In [3]:
import pandas as pd

In [4]:
food_fndds = pd.read_parquet('fndds_parquet/food.parquet')
food_nutrient_fndds = pd.read_parquet('fndds_parquet/food_nutrient.parquet')
nutrient_fndds = pd.read_parquet('fndds_parquet/nutrient.parquet').set_index('nutrient_nbr')
nutrient_fndds = nutrient_fndds.loc[nutrient_fndds.index.notna()]
nl = pd.read_csv('legacy/nutrient.csv')
# nutrient_fndds.drop(columns=['id', 'rank']).to_parquet('fndds_parquet/nutrient.parquet')
# food_fndds.to_parquet('fndds_parquet/food.parquet')
# food_nutrient_fndds.to_parquet('fndds_parquet/food_nutrient.parquet')
# nutrient_fndds.to_parquet('fndds_parquet/nutrient.parquet')

In [5]:
# fnl = food_nutrient_legacy.reset_index().merge(nl[['nutrient_nbr', 'id']], left_on='nutrient_id', right_on='id', suffixes=['', '_n']).drop(columns=['id_n', 'nutrient_id']).rename(columns={'nutrient_nbr': 'nutrient_id'}).set_index('fdc_id')
# fnl = pd.concat([fnl.iloc[:, 0], fnl.iloc[:, -1], fnl.iloc[:, 1:-1]], axis=1)
# fnl.to_parquet('legacy_parquet/food_nutrient.parquet')
# fnl = pd.concat([fnl.iloc[:, 0], fnl.iloc[:, -1], fnl.iloc[:, 1:-1]], axis=1)
# fnl.to_parquet('legacy_parquet/food_nutrient.parquet')

In [6]:
food_legacy = pd.read_parquet('legacy_parquet/food.parquet')
food_nutrient_legacy = pd.read_parquet('legacy_parquet/food_nutrient.parquet')
nutrient_legacy = pd.read_parquet('legacy_parquet/nutrient.parquet')
id_to_nbd = pd.read_parquet('legacy_parquet/id_to_nbd.parquet') 
f_ndb = food_legacy.join(id_to_nbd, how='left').reset_index().set_index('NDB_number')
# food_legacy.to_parquet('legacy_parquet/food.parquet')
# food_nutrient_legacy.to_parquet('legacy_parquet/food_nutrient.parquet')
# nutrient_legacy.to_parquet('legacy_parquet/nutrient.parquet')
# id_to_nbd.to_parquet('legacy_parquet/id_to_nbd.parquet')

In [7]:
codes = pd.read_csv('food_codes.csv', encoding='latin1')
codes = codes.loc[codes.price.notna()]
codes_legacy = codes.loc[codes.fndds_code.isna()].copy()
codes_fndds = codes.loc[codes.fndds_code.notna()].copy()
codes_fndds['fndds_code'] = codes_fndds.fndds_code.astype(int)
codes_legacy = codes_legacy.merge(f_ndb, left_on='ndb_code', right_index=True)
codes_legacy = codes_legacy[['food', 'name', 'price', 'lbs', 'multiplier', 'fdc_id', 'description']]
codes_fndds = codes_fndds.merge(food_fndds.reset_index(), left_on='fndds_code', right_on='food_code')
codes_fndds = codes_fndds[['food', 'name', 'price', 'lbs', 'multiplier', 'fdc_id', 'description']]
codes_fndds = codes_fndds.rename(columns={'food_code': 'fdc_id'})
codes = pd.concat([codes_legacy, codes_fndds]).set_index('fdc_id')

In [8]:
food_nutrient = pd.concat([food_nutrient_fndds, food_nutrient_legacy])
core_nutrients = codes.join(food_nutrient, how='left')
core_nutrients['nutrient_id'] = core_nutrients.nutrient_id.astype(float)
core_nutrients = core_nutrients.drop(columns='name')

In [9]:
cnap = core_nutrients.merge(nutrient_fndds.reset_index(), left_on='nutrient_id', right_on='nutrient_nbr', how='inner')
cna = cnap.loc[cnap.nutrient_id != 268].pivot(index='food', columns=['name','nutrient_nbr'], values='amount')
cna.to_parquet('core_nutrient_amounts_v1.parquet')
cna.droplevel(1, axis=1).sort_values(by='Energy')

name,Folic acid,"Folate, DFE",Betaine,Isoleucine,Leucine,Lysine,Cystine,Valine,Arginine,Histidine,...,Lycopene,"Vitamin B-12, added","Vitamin E, added",Cholesterol,"Folate, total",Vitamin B-12,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),"Fluoride, F",Vitamin D3 (cholecalciferol)
food,,,,,,,,,,,,,,,,,,,,,
Bok Choy,0.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,66.0,0.0,45.0,0.0,NaN,NaN
Lettuce,0.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,34.0,0.0,6.0,0.0,NaN,NaN
Cucumber,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,7.0,0.0,2.8,0.0,NaN,NaN
Zucchini,0.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,24.0,0.0,17.9,0.0,NaN,NaN
Swiss Chard,0.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,14.0,0.0,30.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Almonds,0.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,55.0,0.0,0.0,0.0,NaN,NaN
Hazelnuts,0.0,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,113.0,0.0,6.3,0.0,NaN,NaN
Walnuts,0.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,98.0,0.0,1.3,0.0,NaN,NaN


In [10]:
cnap['price_per_lb'] = cnap.price / cnap.lbs
cnap['multiplier'] = cnap.multiplier.fillna(1)
cnap['price_per_lb_adjusted'] = cnap.price / (cnap.lbs * cnap.multiplier)
cnap = cnap.loc[:,['food', 'price', 'lbs', 'multiplier', 'description', 'amount',
             'nutrient_nbr', 'name',
       'unit_name', 'price_per_lb', 'price_per_lb_adjusted']]
cnap.to_parquet('core_nutrient_amounts_prices_v1.parquet')

In [11]:
cnap.name.unique()

array(['Folic acid', 'Folate, DFE', 'Betaine', 'Isoleucine', 'Leucine',
       'Lysine', 'Cystine', 'Valine', 'Arginine', 'Histidine',
       'Aspartic acid', 'Serine', 'SFA 10:0', 'SFA 16:0', 'SFA 18:0',
       'SFA 20:0', 'MUFA 18:1', 'PUFA 18:2', 'PUFA 18:3', 'PUFA 20:4',
       'PUFA 18:4', 'MUFA 20:1', 'MUFA 22:1',
       'Fatty acids, total monounsaturated',
       'Fatty acids, total polyunsaturated', 'SFA 15:0',
       'PUFA 20:2 n-6 c,c', 'PUFA 20:3', 'PUFA 21:5', 'Total lipid (fat)',
       'Carbohydrate, by difference', 'Energy', 'Sucrose', 'Glucose',
       'Water', 'Sugars, Total', 'Calcium, Ca', 'Potassium, K',
       'Zinc, Zn', 'Selenium, Se', 'Vitamin E (alpha-tocopherol)',
       'Tocopherol, beta', 'Tocopherol, gamma', 'Tocotrienol, alpha',
       'Tocotrienol, beta', 'Niacin', 'Pantothenic acid', 'Vitamin B-6',
       'Protein', 'Ash', 'Starch', 'Fructose', 'Fiber, total dietary',
       'Iron, Fe', 'Magnesium, Mg', 'Phosphorus, P', 'Sodium, Na',
       'Copper, Cu'

In [12]:
def ppnu(df, nutrient):
    return df.loc[df.name == nutrient].sort_values(by='price_per_nutrient_unit')

In [13]:
cnap.unit_name.unique()
LB_CONVERSIONS = {
    'G': 0.00220462,
    'MG': 0.00000220462,
    'UG': 0.00000000220462,
}

In [14]:
cnap['g'] = cnap.lbs * 453.592
cnap['price_per_g'] = cnap.price / cnap.g
cnap['price_per_g_adjusted'] = cnap.price / (cnap.g * cnap.multiplier)
cnap['price_per_100_g'] = cnap.price_per_g * 100
cnap['price_per_100_g_adjusted'] = cnap.price_per_g_adjusted * 100
cnap['price_per_nutrient_unit'] = cnap.price_per_100_g_adjusted / cnap.amount

In [15]:
ppnu(cnap, 'PUFA 20:5 n-3 (EPA)')

,food,price,lbs,multiplier,description,amount,nutrient_nbr,name,unit_name,price_per_lb,price_per_lb_adjusted,g,price_per_g,price_per_g_adjusted,price_per_100_g,price_per_100_g_adjusted,price_per_nutrient_unit
542,Sardines,4.00,1.00,1.00,"Fish, sardine, Atlantic, canned in oil, draine...",0.473,629.0,PUFA 20:5 n-3 (EPA),G,4.000000,4.000000,453.592,0.008818,0.008818,0.881850,0.881850,1.864376
4658,Salmon,8.00,1.00,0.75,"Fish, salmon, NFS",0.860,629.0,PUFA 20:5 n-3 (EPA),G,8.000000,10.666667,453.592,0.017637,0.023516,1.763700,2.351599,2.734418
4733,Pink Salmon,3.60,1.00,1.00,"Fish, salmon, canned",0.274,629.0,PUFA 20:5 n-3 (EPA),G,3.600000,3.600000,453.592,0.007937,0.007937,0.793665,0.793665,2.896587
4973,Mussels,10.00,1.00,1.00,Mussels,0.238,629.0,PUFA 20:5 n-3 (EPA),G,10.000000,10.000000,453.592,0.022046,0.022046,2.204624,2.204624,9.263128
419,Tuna,3.50,1.00,1.00,"Fish, tuna, light, canned in water, without sa...",0.047,629.0,PUFA 20:5 n-3 (EPA),G,3.500000,3.500000,453.592,0.007716,0.007716,0.771619,0.771619,16.417416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,Lettuce,1.00,1.00,1.00,"Lettuce, raw",0.000,629.0,PUFA 20:5 n-3 (EPA),G,1.000000,1.000000,453.592,0.002205,0.002205,0.220462,0.220462,inf
1889,Brussels Sprouts,1.55,0.75,1.00,"Brussels sprouts, raw",0.000,629.0,PUFA 20:5 n-3 (EPA),G,2.066667,2.066667,340.194,0.004556,0.004556,0.455622,0.455622,inf
1806,Bok Choy,1.50,1.00,1.00,"Cabbage, Chinese, raw",0.000,629.0,PUFA 20:5 n-3 (EPA),G,1.500000,1.500000,453.592,0.003307,0.003307,0.330694,0.330694,inf
5951,Whey Protein Powder,10.00,1.00,1.00,Nutritional powder mix (EAS Whey Protein Powder),0.000,629.0,PUFA 20:5 n-3 (EPA),G,10.000000,10.000000,453.592,0.022046,0.022046,2.204624,2.204624,inf


In [16]:
cnap.to_parquet('core_nutrient_amounts_prices_v2.parquet')

In [17]:
constraints= pd.read_csv('nutrient_constraints_csv.csv')
cne = constraints.loc[constraints.target.notna() | constraints.ll.notna() | constraints.ul.notna()]

In [18]:
cnap['nutrient_unit_per_dollar'] = cnap.amount / cnap.price_per_100_g_adjusted

In [19]:
cnap.to_csv('core_nutrient_amounts_prices_v3.csv')